# ElevenLabs Text-to-Speech Instrumentation Example

This notebook demonstrates how to use OpenInference instrumentation with ElevenLabs TTS API, sending traces to Arize.

## Setup

Install required packages:

In [10]:
%pip install elevenlabs arize-otel python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Load Environment Variables

Copy `.env.example` to `.env` and fill in your keys.

In [11]:
from dotenv import load_dotenv

load_dotenv()

True

## Configure Arize OTel

In [12]:
import os

from arize.otel import register

tracer_provider = register(
    space_id=os.environ["ARIZE_SPACE_ID"],
    api_key=os.environ["ARIZE_API_KEY"],
    project_name=os.environ.get("ARIZE_PROJECT", "elevenlabs-demo"),
)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: elevenlabs
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Instrument ElevenLabs

In [13]:
from openinference.instrumentation.elevenlabs import ElevenLabsInstrumentor

ElevenLabsInstrumentor().instrument(tracer_provider=tracer_provider)

## Text-to-Speech: Convert (Sync)

In [14]:
import os

from elevenlabs import ElevenLabs

client = ElevenLabs(api_key=os.environ["ELEVEN_API_KEY"])

# Generate speech - returns audio bytes
audio = client.text_to_speech.convert(
    voice_id="JBFqnCBsd6RMkjVDRZzb",  # George voice
    text="Hello! This is a test of ElevenLabs text to speech.",
    model_id="eleven_multilingual_v2",
)

# Collect audio bytes
audio_bytes = b"".join(audio)
print(f"Generated {len(audio_bytes)} bytes of audio")

Generated 52288 bytes of audio


## Text-to-Speech: Stream (Sync)

In [15]:
# Stream audio chunks
audio_stream = client.text_to_speech.stream(
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    text="This demonstrates streaming audio generation.",
    model_id="eleven_multilingual_v2",
)

chunks = []
for chunk in audio_stream:
    chunks.append(chunk)

print(f"Received {len(chunks)} audio chunks")

Received 43 audio chunks


## Text-to-Speech: Async Convert

In [16]:
from elevenlabs import AsyncElevenLabs

async_client = AsyncElevenLabs(api_key=os.environ["ELEVEN_API_KEY"])


async def async_tts_example():
    # Note: convert() returns an async generator directly, no await needed
    audio = async_client.text_to_speech.convert(
        voice_id="JBFqnCBsd6RMkjVDRZzb",
        text="This is an async text to speech call.",
        model_id="eleven_multilingual_v2",
    )
    audio_bytes = b"".join([chunk async for chunk in audio])
    print(f"Async generated {len(audio_bytes)} bytes of audio")


await async_tts_example()

Async generated 41004 bytes of audio


## Text-to-Speech with Timestamps

In [17]:
# Get audio with character-level timestamps
response = client.text_to_speech.convert_with_timestamps(
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    text="Hello world!",
    model_id="eleven_multilingual_v2",
)

# Response is a single object with audio_base_64 and alignment data
if response.audio_base_64:
    import base64

    audio_bytes = base64.b64decode(response.audio_base_64)
    print(f"Audio received: {len(audio_bytes)} bytes")

if response.alignment:
    print(f"Characters: {response.alignment.characters}")
    print(f"Start times: {response.alignment.character_start_times_seconds}")
    print(f"End times: {response.alignment.character_end_times_seconds}")

Audio received: 19688 bytes
Characters: ['H', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd', '!']
Start times: [0.0, 0.116, 0.174, 0.209, 0.255, 0.337, 0.406, 0.464, 0.557, 0.615, 0.685, 0.801]
End times: [0.116, 0.174, 0.209, 0.255, 0.337, 0.406, 0.464, 0.557, 0.615, 0.685, 0.801, 1.161]


## Cleanup

Uninstrument when done (optional):

In [18]:
ElevenLabsInstrumentor().uninstrument()